## **Lectura 1: introducción a Gurobi python**

**1. Instalación y configuración**

- **Instala Gurobi:** Puedes descargarlo desde el sitio web oficial. Es importante seguir las instrucciones específicas de [instalación](https://www.gurobi.com/downloads/). 
- **Licencia:** Gurobi no es software libre, pero ofrece licencias académicas gratuitas.
- **Instala la interfaz de Python:** Una vez instalado Gurobi, instala gurobipy:

  ```bash
  pip install gurobipy
  ```

**2. Configuración inicial en Python**

Antes de usar Gurobi, necesitas importar su módulo en tu script de Python:

```python
from gurobipy import *
```



**3. Creando un modelo simple**

Vamos a resolver el siguiente problema de programación lineal:

Maximizar:

$Z = 3x + 4y$


Sujeto a:

$1x + 2y ≤ 8$

$3x + 2y ≤ 12$

$x, y ≥ 0$


<b>Construyendo el modelo con gurobipy</b>


In [1]:
from gurobipy import *


# Crea el modelo
m = Model("modelo_simple")

# Crea las variables
x = m.addVar(vtype=GRB.CONTINUOUS, name="x")
y = m.addVar(vtype=GRB.CONTINUOUS, name="y")

# Establece la función objetivo
m.setObjective(3*x + 4*y, GRB.MAXIMIZE)

# Añade restricciones
m.addConstr(x + 2*y <= 8, "restriccion_1")
m.addConstr(3*x + 2*y <= 12, "restriccion_2")

# Resuelve el modelo
m.optimize()

# Imprime las soluciones
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))

print('Objetivo: %g' % m.objVal)

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-1265U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x98e9963f
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [3e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 1e+01]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0000000e+30   2.750000e+30   7.000000e+00      0s
       2    1.8000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.800000000e+01
x 2
y 3
Objetivo: 18


**Problema de la dieta**

Hay 5 tipos de alimentos y 2 requerimientos de nutrientes que debemos satisfacer al mínimo coste. Se nos da el contenido nutricional y el coste por onza de cada tipo de alimento.

### Unidades de nutrientes y coste por onza

| Food type | Iron | Calcium | Cost |
|-----------|------|---------|------|
| 1         | 2    | 0       | 20   |
| 2         | 0    | 1       | 10   |
| 3         | 3    | 2       | 31   |
| 4         | 1    | 2       | 11   |
| 5         | 2    | 1       | 12   |


> ¿Cuál es la combinación de costo mínimo de estos alimentos que proporciona al menos 21 unidades de hierro y 12 unidades de calcio?

<b> Formulación del problema de la dieta </b>


- Variables de decisión: 

$x_j$ = número de onzas del tipo de alimento $j=1,\ldots,5$.

 Función Objetivo: 

 min $z = 20 x_1 + 10 x_2 + 31 x_3 + 11 x_4 + 12 x_5$


 Restricciones estructurales:

 $2 x_1 + 0 x_2 + 3 x_3 + 1 x_4 + 2 x_5 \ge 21$ (requisito de hierro) 
 
 $0 x_1 + 1 x_2 + 2 x_3 + 2 x_4 + 1 x_5 \ge 12$ (necesidad de calcio)


Restricciones de no negatividad
 $x_j \ge 0,j=1,\ldots,5$


<b>Construyendo el modelo con gurobipy</b>

Primero necesitaremos importar el módulo gurobipy. Nuestro alias preferido es 'grb'. También crearemos un alias para gurobipy.GRB, que contiene algunas constantes útiles que normalmente necesitaremos.

In [2]:
import gurobipy as grb
GRB = grb.GRB

El objeto gurobipy.Model sirve como repositorio de todos los datos relativos a su problema de optimización, y proporciona métodos para establecer variables de decisión y restricciones.

In [3]:
m = grb.Model()

Puedes llamar a tu objeto gurobipy.Model como quieras, pero aquí hemos elegido "m". Este objeto proporciona métodos para instanciar variables de decisión (addVar) y restricciones estructurales (addConstr).

<b>Añadir Variables</b>

Primero añadiremos las variables de decisión utilizando el método Model.addVar().

In [4]:
?m.addVar

Signature: m.addVar(lb=0.0, ub=1e+100, obj=0.0, vtype='C', name='', column=None)
Docstring:
ROUTINE:
  addVar(lb, ub, obj, vtype, name, column)

PURPOSE:
  Add a variable to the model.

ARGUMENTS:
  lb (float): Lower bound (default is zero)
  ub (float): Upper bound (default is infinite)
  obj (float): Objective coefficient (default is zero)
  vtype (string): Variable type (default is GRB.CONTINUOUS)
  name (string): Variable name (default is no name)
  column (Column): Initial coefficients for column (default is None)

RETURN VALUE:
  The created Var object.

EXAMPLE:
  v = model.addVar(ub=2.0, name="NewVar")
Type:      method

Este método toma una variedad de parámetros opcionales y devuelve una referencia a un objeto `gurobipy.Var`. Necesitará guardar estas referencias para construir restricciones estructurales más adelante. Si conoce los límites inferiores o superiores de las variables, esos límites deben pasarse a través de los argumentos `lb` y `ub`, que por defecto son 0 e infinito, respectivamente. También en este punto tiene la opción de especificar un coeficiente objetivo a través del argumento `obj`. 

Es opcional, pero en general es una buena práctica proporcionar nombres significativos para cada una de las variables. En última instancia vamos a querer la salida y ver el modelo que hemos construido para verificar que es lo que pretendíamos, y los nombres que ha establecido aquí aparecerá en la salida. Esos nombres no necesitan coincidir con los nombres de las variables Python usadas para establecer las variables, lo que significa que no necesitan seguir las reglas para nombrar variables Python. Una buena convención para nombrarlas es usar una palabra o frase corta que describa lo que representa la variable, seguida de una lista separada por puntos de los índices de la variable (en este caso los tipos de alimentos del 1 al 5).

In [5]:
x1 = m.addVar(lb=0, ub=GRB.INFINITY, obj=20, vtype=GRB.CONTINUOUS, name='consumed.1')

In [6]:
x2 = m.addVar(obj=10, name='consumed.2')

In [7]:
x3 = m.addVar(obj=31, name='consumed.3')

In [8]:
x4 = m.addVar(obj=11, name='consumed.4')

In [9]:
x5 = m.addVar(obj=12, name='consumed.5')

In [10]:
# At this point we can print what we have saved from the calls to Model.addVar().
print( x1, x2, x3, x4, x5)

<gurobi.Var *Awaiting Model Update*> <gurobi.Var *Awaiting Model Update*> <gurobi.Var *Awaiting Model Update*> <gurobi.Var *Awaiting Model Update*> <gurobi.Var *Awaiting Model Update*>


In [11]:
m.update()
print( x1, x2, x3, x4, x5)

<gurobi.Var consumed.1> <gurobi.Var consumed.2> <gurobi.Var consumed.3> <gurobi.Var consumed.4> <gurobi.Var consumed.5>


<b>Construcción de expresiones lineales</b>

Las restricciones estructurales implican múltiples variables de decisión. El objeto `gurobipy.LinExpr` almacena una función lineal de variables de decisión. Los operadores + y * están sobrecargados para que pueda crear expresiones lineales de forma natural.

In [12]:
lhs = 2*x1 + 3*x3 + x4 + 2*x5
print (lhs)

2.0 consumed.1 + 3.0 consumed.3 + consumed.4 + 2.0 consumed.5


<b>Añadir Restricciones Estructurales</b>

Para añadir una restricción estructural, llame al método addConstr en un objeto Modelo.

In [13]:
?m.addConstr

Signature: m.addConstr(lhs, sense=None, rhs=None, name='')
Docstring:
ROUTINE:
  addConstr(tc, name)

PURPOSE:
  Add a constraint to the model.

ARGUMENTS:
  tc (TempConstr): The constraint to add
  name (string): Constraint name (default is no name)

RETURN VALUE:
  Depending on the data of 'tc':
    - A Constr object if tc arose from a linear expression
    - A QConstr object if tc arose from a quadratic expression
    - An MConstr object if tc arose from a linear matrix expression
    - A GenConstr object if tc arose form a general constraint expression

EXAMPLE:
  c = model.addConstr(x + y <= 1)
  c = model.addConstr(x + y + z == [1, 2])
  c = model.addConstr(x*x + y*y <= 1)
  c = model.addConstr(z == and_(y, x, w))
  c = model.addConstr(z == min_(x, y))
  c = model.addConstr((w == 1) >> (x + y <= 1))
  c = model.addConstr(A @ x <= b)
Type:      method

Vamos a demostrar dos maneras de llamar `Model.addConstr()` aquí. Preferimos la segunda opción, ya que tiene una conexión más explícita con la restricción en el modelo anterior.

In [14]:
iron_constr = m.addConstr(lhs, '>', 21, name='nutrient.iron')

In [15]:
x2 + 2*x3 + 2*x4 + x5 >= 12

<gurobi.TempConstr: consumed.2 + 2.0 consumed.3 + 2.0 consumed.4 + consumed.5 >= 12>

También se han sobrecargado los operadores ==, >= y <= para que las restricciones puedan escribirse de forma natural.

In [16]:
calcium_constr = m.addConstr(x2 + 2*x3 + 2*x4 + x5 >= 12, name='nutrient.calcium')

Al igual que con las variables, es una buena práctica seguir una convención de nomenclatura coherente con las restricciones. Estos nombres le ayudarán a identificar las restricciones cuando visualice el modelo completo.

Una vez creadas todas las restricciones, pueden añadirse al modelo por lotes llamando al método update.

In [17]:
m.update()

<b>Inspección del modelo</b>

Podemos optimizar en este punto, pero es bueno tener alguna garantía de que el código que acabamos de escribir realmente construyó el modelo que esperamos que construya. El formato de archivo .lp proporciona una representación legible del modelo.

In [18]:
m.write('diet.lp')

In [19]:
!more diet.lp

\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  20 consumed.1 + 10 consumed.2 + 31 consumed.3 + 11 consumed.4
   + 12 consumed.5
Subject To
 nutrient.iron: 2 consumed.1 + 3 consumed.3 + consumed.4 + 2 consumed.5
   >= 21
 nutrient.calcium: consumed.2 + 2 consumed.3 + 2 consumed.4 + consumed.5
   >= 12
Bounds
End


<b>Resolución del modelo</b>

Si se ve bien, estamos listos para resolver el modelo, lo que hacemos llamando a Model.optimize(). Después, podemos comprobar el atributo Status del modelo para ver si la solución ha tenido éxito. Si el estado es igual a GRB.OPTIMAL, entonces Gurobi fue capaz de encontrar una solución óptima demostrable (dentro de una tolerancia).

In [20]:
m.optimize()
print( "Model status =", m.Status)
assert m.Status == GRB.OPTIMAL

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-1265U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2 rows, 5 columns and 8 nonzeros
Model fingerprint: 0x7d44349f
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve removed 0 rows and 3 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.650000e+01   0.000000e+00      0s
       2    1.3100000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.310000000e+02
Model status = 2


La llamada a `Model.optimize()` produce un registro que da varias estadísticas del modelo como el número de variables, restricciones y entradas no nulas de la matriz de restricciones, así como rangos para todas las entradas numéricas. Para soluciones más largas, el registro también incluirá una tabla en la que Gurobi escribe periódicamente para mantenerle informado del progreso. Si se encuentra una solución óptima, el registro terminará con el valor objetivo óptimo.

<b>Consulta de la solución</b>

El registro proporciona un valor objetivo óptimo, pero no los valores óptimos de las variables, que suelen ser la salida más importante de la solución, ya que son los que en última instancia impulsan sus decisiones. Para obtener los datos de la solución, podemos consultar varios atributos de los objetos Var y Constr que guardamos anteriormente.

Aquí veremos dos de estos atributos `Var`. El primer atributo, y normalmente el más importante, se llama simplemente 'X' y proporciona el valor de la variable en la solución óptima. Para el problema de la dieta, esto nos dice el número de onzas de cada tipo de alimento que debemos incluir en nuestra dieta. Podemos utilizar las referencias Var guardadas anteriormente para consultar este atributo.

In [21]:
print(x1.VarName, x1.X)
print(x2.VarName, x2.X)
print(x3.VarName, x3.X)
print(x4.VarName, x4.X)
print(x5.VarName, x5.X)

consumed.1 0.0
consumed.2 0.0
consumed.3 0.0
consumed.4 1.0
consumed.5 10.0


El segundo atributo Var que veremos aquí es el coste reducido, que tiene el nombre de atributo 'RC'. Este coste reducido nos indica la cantidad que tendría que cambiar el coeficiente objetivo de una variable para que ésta tuviera un valor positivo en una solución óptima. Para las variables x.4 y x.5 que ya son positivas, el coste reducido es siempre cero.

In [22]:
print(x1.VarName, x1.RC)
print(x2.VarName, x2.RC)
print(x3.VarName, x3.RC)
print(x4.VarName, x4.RC)
print(x5.VarName, x5.RC)

consumed.1 11.333333333333334
consumed.2 6.666666666666666
consumed.3 11.333333333333332
consumed.4 0.0
consumed.5 0.0


La interpretación aquí es que el tipo de alimento 1 tendría que ser al menos $11 \frac{1}{3}$  más barato para que nos resultara rentable incluirlo en nuestra dieta.

Puede iterar sobre todas las variables del modelo con el método Model.getVars().

In [23]:
for var in m.getVars():
    print(var.VarName, var.X, var.RC)

consumed.1 0.0 11.333333333333334
consumed.2 0.0 6.666666666666666
consumed.3 0.0 11.333333333333332
consumed.4 1.0 0.0
consumed.5 10.0 0.0


El valor objetivo aparece en el registro de soluciones, pero puede consultarse a través del atributo ObjVal del modelo.

In [24]:
print(m.ObjVal)

131.0


También hay información de solución asociada a las restricciones que puede ser útil para algunas aplicaciones. Aquí veremos los atributos Slack y Pi del objeto Constr.

In [25]:
iron_constr.Slack, iron_constr.Pi

(0.0, 4.333333333333333)

In [26]:
for constr in m.getConstrs():
    print(constr.ConstrName, constr.Slack, constr.Pi)

nutrient.iron 0.0 4.333333333333333
nutrient.calcium 0.0 3.333333333333334


El atributo Slack nos da la diferencia entre los lados derecho e izquierdo de la restricción en la solución óptima. El atributo Pi también se conoce como valor dual o precio sombra, y nos dice cuánto cambiaría el objetivo si el lado derecho de la restricción se incrementara en 1 unidad. Una restricción puede tener una holgura positiva o un valor dual positivo, pero nunca ambos simultáneamente positivos. 

En este caso, ninguna de las dos restricciones tiene holgura y ambas tienen valores duales positivos. La interpretación para el hierro sería que el aumento de la necesidad de hierro en 1 unidad (lo que aumenta el lado derecho de la restricción de hierro en 1 unidad) aumentaría el coste óptimo de nuestra dieta en $4 \frac{1}{3}$. Esto significa que el precio máximo que estaríamos dispuestos a pagar por una unidad de hierro debería ser de $4.



Nuestras restricciones estructurales suelen imponer algún tipo de requisito mínimo de recursos para que funcione un sistema. En este caso, las variables duales pueden interpretarse como el precio de esos recursos.

In [27]:
x1.LB = 0.1

In [28]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-1265U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2 rows, 5 columns and 8 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e-01, 1e-01]
  RHS range        [1e+01, 2e+01]
       0    1.3213333e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.321333333e+02


In [29]:
m.ObjVal - 131

1.1333333333333258

In [30]:
iron_constr.RHS

21.0

In [31]:
iron_constr.RHS = 21.1

In [32]:
old_obj = m.ObjVal

In [33]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-1265U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2 rows, 5 columns and 8 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e-01, 1e-01]
  RHS range        [1e+01, 2e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3256667e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.325666667e+02


In [34]:
m.ObjVal - old_obj

0.4333333333333371